In [1]:
"""
You can run either this notebook locally (if you have all the dependencies and a GPU) or on Google Colab.

Instructions for setting up Colab are as follows:
1. Open a new Python 3 notebook.
2. Import this notebook from GitHub (File -> Upload Notebook -> "GITHUB" tab -> copy/paste GitHub URL)
3. Connect to an instance with a GPU (Runtime -> Change runtime type -> select "GPU" for hardware accelerator)
4. Run this cell to set up dependencies.
"""
# If you're using Google Colab and not running locally, run this cell.

## Install dependencies
!pip install wget
!apt-get install sox libsndfile1 ffmpeg
!pip install unidecode

# ## Install NeMo
BRANCH = 'r1.4.0'
# !python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[asr]

## Install TorchAudio
!pip install torchaudio>=0.6.0 -f https://download.pytorch.org/whl/torch_stable.html

## Grab the config we'll use in this example
!mkdir configs

Reading package lists... Done
Building dependency tree       
Reading state information... Done
ffmpeg is already the newest version (7:4.2.4-1ubuntu0.1).
libsndfile1 is already the newest version (1.0.28-7ubuntu0.1).
sox is already the newest version (14.4.2+git20190427-2).
0 upgraded, 0 newly installed, 0 to remove and 0 not upgraded.


# Introduction

This Speech Command recognition tutorial is based on the MatchboxNet model from the paper ["MatchboxNet: 1D Time-Channel Separable Convolutional Neural Network Architecture for Speech Commands Recognition"](https://arxiv.org/abs/2004.08531). MatchboxNet is a modified form of the QuartzNet architecture from the paper "[QuartzNet: Deep Automatic Speech Recognition with 1D Time-Channel Separable Convolutions](https://arxiv.org/pdf/1910.10261.pdf)" with a modified decoder head to suit classification tasks.

The notebook will follow the steps below:

 - Dataset preparation: Preparing Google Speech Commands dataset

 - Audio preprocessing (feature extraction): signal normalization, windowing, (log) spectrogram (or mel scale spectrogram, or MFCC)

 - Data augmentation using SpecAugment "[SpecAugment: A Simple Data Augmentation Method for Automatic Speech Recognition](https://arxiv.org/abs/1904.08779)" to increase the number of data samples.
 
 - Develop a small Neural classification model that can be trained efficiently.
 
 - Model training on the Google Speech Commands dataset in NeMo.
 
 - Evaluation of error cases of the model by audibly hearing the samples

In [2]:
# Some utility imports
import os
from omegaconf import OmegaConf

In [3]:
# This is where the Google Speech Commands directory will be placed.
# Change this if you don't want the data to be extracted in the current directory.
# Select the version of the dataset required as well (can be 1 or 2)
DATASET_VER = 1
data_dir = './google_dataset_v{0}/'.format(DATASET_VER)

if DATASET_VER == 1:
  MODEL_CONFIG = "matchboxnet_3x1x64_v1.yaml"
else:
  MODEL_CONFIG = "matchboxnet_3x1x64_v2.yaml"

if not os.path.exists(f"configs/{MODEL_CONFIG}"):
  !wget -P configs/ "https://raw.githubusercontent.com/NVIDIA/NeMo/$BRANCH/examples/asr/conf/matchboxnet/{MODEL_CONFIG}"

--2021-11-29 00:54:43--  https://raw.githubusercontent.com/NVIDIA/NeMo/r1.4.0/examples/asr/conf/matchboxnet/matchboxnet_3x1x64_v1.yaml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4652 (4.5K) [text/plain]
Saving to: ‘configs/matchboxnet_3x1x64_v1.yaml’

matchboxnet_3x1x64_ 100%[===================>]   4.54K  --.-KB/s    in 0s      

2021-11-29 00:54:43 (63.8 MB/s) - ‘configs/matchboxnet_3x1x64_v1.yaml’ saved [4652/4652]



# Data Preparation

We will be using the open-source Google Speech Commands Dataset (we will use V1 of the dataset for the tutorial but require minor changes to support the V2 dataset). These scripts below will download the dataset and convert it to a format suitable for use with NeMo.

## Download the dataset

The dataset must be prepared using the scripts provided under the `{NeMo root directory}/scripts` sub-directory. 

Run the following command below to download the data preparation script and execute it.

**NOTE**: You should have at least 4GB of disk space available if you’ve used --data_version=1; and at least 6GB if you used --data_version=2. Also, it will take some time to download and process, so go grab a coffee.

**NOTE**: You may additionally pass a `--rebalance` flag at the end of the `process_speech_commands_data.py` script to rebalance the class samples in the manifest.

In [4]:
if not os.path.exists("process_speech_commands_data.py"):
  !wget https://raw.githubusercontent.com/NVIDIA/NeMo/$BRANCH/scripts/dataset_processing/process_speech_commands_data.py

### Preparing the manifest file

The manifest file is a simple file that has the full path to the audio file, the duration of the audio file, and the label that is assigned to that audio file. 

This notebook is only a demonstration, and therefore we will use the `--skip_duration` flag to speed up construction of the manifest file.

**NOTE: When replicating the results of the paper, do not use this flag and prepare the manifest file with correct durations.**

In [5]:
!mkdir {data_dir}
!python process_speech_commands_data.py --data_root={data_dir} --data_version={DATASET_VER} --skip_duration --log
print("Dataset ready !")

mkdir: cannot create directory ‘./google_dataset_v1/’: File exists
INFO:root:Working on: google_speech_recognition_v1
INFO:root:Getting google_speech_recognition_v1
INFO:root:./google_dataset_v1/google_speech_recognition_v1.tar.bz2 does not exist. Downloading ...
INFO:root:Downloaded ./google_dataset_v1/google_speech_recognition_v1.tar.bz2.
INFO:root:Extracting google_speech_recognition_v1
INFO:root:Processing google_speech_recognition_v1
INFO:root:Validation and test set lists extracted
INFO:root:Prepared filepaths for dataset
INFO:root:Preparing manifest : train_manifest.json with #51088 files
INFO:root:Finished construction of manifest : train_manifest.json
INFO:root:
<<NOTE>> Duration computation was skipped for demonstration purposes on Colaboratory.
In order to replicate paper results and properly perform data augmentation, 
please recompute the manifest file without the `--skip_duration` flag !

INFO:root:Preparing manifest : validation_manifest.json with #6798 files
INFO:root:F

## Prepare the path to manifest files

In [6]:
dataset_path = 'google_speech_recognition_v{0}'.format(DATASET_VER)
dataset_basedir = os.path.join(data_dir, dataset_path)

train_dataset = os.path.join(dataset_basedir, 'train_manifest.json')
val_dataset = os.path.join(dataset_basedir, 'validation_manifest.json')
test_dataset = os.path.join(dataset_basedir, 'test_manifest.json')

## Read a few rows of the manifest file 

Manifest files are the data structure used by NeMo to declare a few important details about the data :

1) `audio_filepath`: Refers to the path to the raw audio file <br>
2) `command`: The class label (or speech command) of this sample <br>
3) `duration`: The length of the audio file, in seconds.

In [7]:
!head -n 5 {train_dataset}

{"audio_filepath": "./google_dataset_v1/google_speech_recognition_v1/right/24befdb3_nohash_0.wav", "duration": 0.0, "command": "right"}
{"audio_filepath": "./google_dataset_v1/google_speech_recognition_v1/right/353b4d33_nohash_0.wav", "duration": 0.0, "command": "right"}
{"audio_filepath": "./google_dataset_v1/google_speech_recognition_v1/right/6f342826_nohash_0.wav", "duration": 0.0, "command": "right"}
{"audio_filepath": "./google_dataset_v1/google_speech_recognition_v1/right/5aac2efa_nohash_2.wav", "duration": 0.0, "command": "right"}
{"audio_filepath": "./google_dataset_v1/google_speech_recognition_v1/right/493392c6_nohash_0.wav", "duration": 0.0, "command": "right"}


In [8]:
!cat {train_dataset} | wc -l

51088


In [9]:
!cat {val_dataset} | wc -l

6798


In [10]:
!cat {test_dataset} | wc -l

6835


# Training - Preparation

We will be training a MatchboxNet model from the paper ["MatchboxNet: 1D Time-Channel Separable Convolutional Neural Network Architecture for Speech Commands Recognition"](https://arxiv.org/abs/2004.08531). The benefit of MatchboxNet over JASPER models is that they use 1D Time-Channel Separable Convolutions, which greatly reduce the number of parameters required to obtain good model accuracy.

MatchboxNet models generally follow the model definition pattern QuartzNet-[BxRXC], where B is the number of blocks, R is the number of convolutional sub-blocks, and C is the number of channels in these blocks. Each sub-block contains a 1-D masked convolution, batch normalization, ReLU, and dropout.

An image of QuartzNet, the base configuration of MatchboxNet models, is provided below.


<p align="center">
  <img src="https://developer.nvidia.com/blog/wp-content/uploads/2020/05/quartznet-model-architecture-1-625x742.png">
</p>

In [11]:
# NeMo's "core" package
import nemo
# NeMo's ASR collection - this collections contains complete ASR models and
# building blocks (modules) for ASR
import nemo.collections.asr as nemo_asr

################################################################################
### WARNING, path does not exist: KALDI_ROOT=/mnt/matylda5/iveselyk/Tools/kaldi-trunk
###          (please add 'export KALDI_ROOT=<your_path>' in your $HOME/.profile)
###          (or run as: KALDI_ROOT=<your_path> python <your_script>.py)
################################################################################

[NeMo W 2021-11-29 00:55:29 experimental:27] Module <class 'nemo.collections.asr.data.audio_to_text_dali._AudioTextDALIDataset'> is experimental, not ready for production and is not fully supported. Use at your own risk.


## Model Configuration
The MatchboxNet Model is defined in a config file which declares multiple important sections.

They are:

1) `model`: All arguments that will relate to the Model - preprocessors, encoder, decoder, optimizer and schedulers, datasets and any other related information

2) `trainer`: Any argument to be passed to PyTorch Lightning

In [12]:
# This line will print the entire config of the MatchboxNet model
config_path = f"configs/{MODEL_CONFIG}"
config = OmegaConf.load(config_path)
config = OmegaConf.to_container(config, resolve=True)
config = OmegaConf.create(config)
print(OmegaConf.to_yaml(config))

name: MatchboxNet-3x1x64-v1
model:
  sample_rate: 16000
  timesteps: 128
  repeat: 1
  dropout: 0.0
  kernel_size_factor: 1.0
  labels_full:
  - bed
  - bird
  - cat
  - dog
  - down
  - eight
  - five
  - four
  - go
  - happy
  - house
  - left
  - marvin
  - nine
  - 'no'
  - 'off'
  - 'on'
  - one
  - right
  - seven
  - sheila
  - six
  - stop
  - three
  - tree
  - two
  - up
  - wow
  - 'yes'
  - zero
  labels_subset:
  - 'yes'
  - 'no'
  - up
  - down
  - left
  - right
  - 'on'
  - 'off'
  - stop
  - go
  - unknown
  - silence
  labels:
  - bed
  - bird
  - cat
  - dog
  - down
  - eight
  - five
  - four
  - go
  - happy
  - house
  - left
  - marvin
  - nine
  - 'no'
  - 'off'
  - 'on'
  - one
  - right
  - seven
  - sheila
  - six
  - stop
  - three
  - tree
  - two
  - up
  - wow
  - 'yes'
  - zero
  train_ds:
    manifest_filepath: ???
    sample_rate: 16000
    labels:
    - bed
    - bird
    - cat
    - dog
    - down
    - eight
    - five
    - four
    - go
    - ha

In [13]:
# Preserve some useful parameters
labels = config.model.labels
sample_rate = config.model.sample_rate

### Setting up the datasets within the config

If you'll notice, there are a few config dictionaries called `train_ds`, `validation_ds` and `test_ds`. These are configurations used to setup the Dataset and DataLoaders of the corresponding config.



In [14]:
print(OmegaConf.to_yaml(config.model.train_ds))

manifest_filepath: ???
sample_rate: 16000
labels:
- bed
- bird
- cat
- dog
- down
- eight
- five
- four
- go
- happy
- house
- left
- marvin
- nine
- 'no'
- 'off'
- 'on'
- one
- right
- seven
- sheila
- six
- stop
- three
- tree
- two
- up
- wow
- 'yes'
- zero
batch_size: 128
shuffle: true
is_tarred: false
tarred_audio_filepaths: null
tarred_shard_strategy: scatter
augmentor:
  shift:
    prob: 1.0
    min_shift_ms: -5.0
    max_shift_ms: 5.0
  white_noise:
    prob: 1.0
    min_level: -90
    max_level: -46



### `???` inside configs

You will often notice that some configs have `???` in place of paths. This is used as a placeholder so that the user can change the value at a later time.

Let's add the paths to the manifests to the config above.

In [15]:
config.model.train_ds.manifest_filepath = train_dataset
config.model.validation_ds.manifest_filepath = val_dataset
config.model.test_ds.manifest_filepath = test_dataset

## Building the PyTorch Lightning Trainer

NeMo models are primarily PyTorch Lightning modules - and therefore are entirely compatible with the PyTorch Lightning ecosystem!

Lets first instantiate a Trainer object!

In [16]:
import torch
import pytorch_lightning as pl

In [17]:
print("Trainer config - \n")
print(OmegaConf.to_yaml(config.trainer))

Trainer config - 

gpus: 0
max_epochs: 200
max_steps: null
num_nodes: 1
accelerator: ddp
accumulate_grad_batches: 1
checkpoint_callback: false
logger: false
log_every_n_steps: 1
val_check_interval: 1.0



In [18]:
# Lets modify some trainer configs for this demo
# Checks if we have GPU available and uses it
cuda = 1 if torch.cuda.is_available() else 0
config.trainer.gpus = cuda

# Reduces maximum number of epochs to 5 for quick demonstration
config.trainer.max_epochs = 5

# Remove distributed training flags
config.trainer.accelerator = None

In [19]:
trainer = pl.Trainer(**config.trainer)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


## Setting up a NeMo Experiment

NeMo has an experiment manager that handles logging and checkpointing for us, so let's use it ! 

In [20]:
from nemo.utils.exp_manager import exp_manager

In [21]:
exp_dir = exp_manager(trainer, config.get("exp_manager", None))

[NeMo I 2021-11-29 00:55:29 exp_manager:220] Experiments will be logged at /workspace/nemo/tutorials/asr/nemo_experiments/MatchboxNet-3x1x64-v1/2021-11-29_00-55-29
[NeMo I 2021-11-29 00:55:29 exp_manager:569] TensorboardLogger has been set up


[NeMo W 2021-11-29 00:55:29 nemo_logging:349] /opt/conda/lib/python3.8/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:240: LightningDeprecationWarning: `ModelCheckpoint(every_n_val_epochs)` is deprecated in v1.4 and will be removed in v1.6. Please use `every_n_epochs` instead.
      rank_zero_deprecation(
    


In [22]:
# The exp_dir provides a path to the current experiment for easy access
exp_dir = str(exp_dir)
exp_dir

'/workspace/nemo/tutorials/asr/nemo_experiments/MatchboxNet-3x1x64-v1/2021-11-29_00-55-29'

## Building the MatchboxNet Model

MatchboxNet is an ASR model with a classification task - it generates one label for the entire provided audio stream. Therefore we encapsulate it inside the `EncDecClassificationModel` as follows.

In [23]:
asr_model = nemo_asr.models.EncDecClassificationModel(cfg=config.model, trainer=trainer)

[NeMo I 2021-11-29 00:55:29 collections:266] Filtered duration for loading collection is 0.000000.
[NeMo I 2021-11-29 00:55:29 collections:270] # 51088 files loaded accounting to # 30 labels
[NeMo I 2021-11-29 00:55:29 collections:266] Filtered duration for loading collection is 0.000000.
[NeMo I 2021-11-29 00:55:29 collections:270] # 6798 files loaded accounting to # 30 labels
[NeMo I 2021-11-29 00:55:30 collections:266] Filtered duration for loading collection is 0.000000.
[NeMo I 2021-11-29 00:55:30 collections:270] # 6835 files loaded accounting to # 30 labels


# Training a MatchboxNet Model

As MatchboxNet is inherently a PyTorch Lightning Model, it can easily be trained in a single line - `trainer.fit(model)` !

### Monitoring training progress

Before we begin training, let's first create a Tensorboard visualization to monitor progress


In [24]:
try:
  from google import colab
  COLAB_ENV = True
except (ImportError, ModuleNotFoundError):
  COLAB_ENV = False

In [25]:
# Load the TensorBoard notebook extension
if COLAB_ENV:
  %load_ext tensorboard
else:
  print("To use tensorboard, please use this notebook in a Google Colab environment.")

To use tensorboard, please use this notebook in a Google Colab environment.


In [26]:
if COLAB_ENV:
  %tensorboard --logdir {exp_dir}
else:
  print("To use tensorboard, please use this notebook in a Google Colab environment.")

To use tensorboard, please use this notebook in a Google Colab environment.


In [27]:
%load_ext tensorboard
%tensorboard --logdir {exp_dir}

### Training for 5 epochs
We see below that the model begins to get modest scores on the validation set after just 5 epochs of training

In [28]:
%%time
trainer.fit(asr_model)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


[NeMo I 2021-11-29 00:55:33 modelPT:544] Optimizer config = Novograd (
    Parameter Group 0
        amsgrad: False
        betas: [0.95, 0.5]
        eps: 1e-08
        grad_averaging: False
        lr: 0.05
        weight_decay: 0.001
    )
[NeMo I 2021-11-29 00:55:33 lr_scheduler:621] Scheduler "<nemo.core.optim.lr_scheduler.PolynomialHoldDecayAnnealing object at 0x7f736c4b04c0>" 
    will be used during training (effective maximum steps = 2000) - 
    Parameters : 
    (power: 2.0
    warmup_ratio: 0.05
    hold_ratio: 0.45
    min_lr: 0.001
    last_epoch: -1
    max_steps: 2000
    )



  | Name              | Type                             | Params
-----------------------------------------------------------------------
0 | spec_augmentation | SpectrogramAugmentation          | 0     
1 | crop_or_pad       | CropOrPadSpectrogramAugmentation | 0     
2 | preprocessor      | AudioToMFCCPreprocessor          | 0     
3 | encoder           | ConvASREncoder                   | 73.3 K
4 | decoder           | ConvASRDecoderClassification     | 3.9 K 
5 | loss              | CrossEntropyLoss                 | 0     
6 | _accuracy         | TopKClassificationAccuracy       | 0     
-----------------------------------------------------------------------
77.2 K    Trainable params
0         Non-trainable params
77.2 K    Total params
0.309     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

[NeMo W 2021-11-29 00:55:33 nemo_logging:349] /opt/conda/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:105: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
      rank_zero_warn(
    
[NeMo W 2021-11-29 00:55:36 nemo_logging:349] /opt/conda/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:105: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
      rank_zero_warn(
    


...

Validating: 0it [00:00, ?it/s]

Epoch 3, global step 1599: val_loss reached 0.23915 (best 0.23915), saving model to "/workspace/nemo/tutorials/asr/nemo_experiments/MatchboxNet-3x1x64-v1/2021-11-29_00-55-29/checkpoints/MatchboxNet-3x1x64-v1--val_loss=0.2391-epoch=3.ckpt" as top 3


Validating: 0it [00:00, ?it/s]

Epoch 4, global step 1999: val_loss reached 0.20314 (best 0.20314), saving model to "/workspace/nemo/tutorials/asr/nemo_experiments/MatchboxNet-3x1x64-v1/2021-11-29_00-55-29/checkpoints/MatchboxNet-3x1x64-v1--val_loss=0.2031-epoch=4.ckpt" as top 3


CPU times: user 6min 1s, sys: 14 s, total: 6min 15s
Wall time: 6min 11s


### Evaluation on the Test set

Lets compute the final score on the test set via `trainer.test(model)`

In [29]:
trainer.test(asr_model, ckpt_path=None)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
[NeMo W 2021-11-29 01:01:42 nemo_logging:349] /opt/conda/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:105: UserWarning: The dataloader, test dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
      rank_zero_warn(
    


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_epoch_top@1': 0.9439648389816284, 'test_loss': 0.1982232630252838}
--------------------------------------------------------------------------------


[{'test_loss': 0.1982232630252838, 'test_epoch_top@1': 0.9439648389816284}]

# Fast Training

We can dramatically improve the time taken to train this model by using Multi GPU training along with Mixed Precision.

For multi-GPU training, take a look at [the PyTorch Lightning Multi-GPU training section](https://pytorch-lightning.readthedocs.io/en/latest/advanced/multi_gpu.html)

For mixed-precision training, take a look at [the PyTorch Lightning Mixed-Precision training section](https://pytorch-lightning.readthedocs.io/en/latest/guides/speed.html#mixed-precision-16-bit-training)

```python
# Mixed precision:
trainer = Trainer(amp_level='O1', precision=16)

# Trainer with a distributed backend:
trainer = Trainer(gpus=2, num_nodes=2, accelerator='ddp')

# Of course, you can combine these flags as well.
```

# Evaluation of incorrectly predicted samples

Given that we have a trained model, which performs reasonably well, let's try to listen to the samples where the model is least confident in its predictions.

For this, we need the support of the librosa library.

**NOTE**: The following code depends on librosa. To install it, run the following code block first.

In [30]:
!pip install librosa

## Extract the predictions from the model

We want to possess the actual logits of the model instead of just the final evaluation score, so we can define a function to perform the forward step for us without computing the final loss. Instead, we extract the logits per batch of samples provided.

## Accessing the data loaders

We can utilize the `setup_test_data` method in order to instantiate a data loader for the dataset we want to analyze.

For convenience, we can access these instantiated data loaders using the following accessors - `asr_model._train_dl`, `asr_model._validation_dl` and `asr_model._test_dl`.

In [31]:
asr_model.setup_test_data(config.model.test_ds)
test_dl = asr_model._test_dl

[NeMo I 2021-11-29 01:01:47 collections:266] Filtered duration for loading collection is 0.000000.
[NeMo I 2021-11-29 01:01:47 collections:270] # 6835 files loaded accounting to # 30 labels


## Partial Test Step

Below we define a utility function to perform most of the test step. For reference, the test step is defined as follows:

```python
    def test_step(self, batch, batch_idx, dataloader_idx=0):
        audio_signal, audio_signal_len, labels, labels_len = batch
        logits = self.forward(input_signal=audio_signal, input_signal_length=audio_signal_len)
        loss_value = self.loss(logits=logits, labels=labels)
        correct_counts, total_counts = self._accuracy(logits=logits, labels=labels)
        return {'test_loss': loss_value, 'test_correct_counts': correct_counts, 'test_total_counts': total_counts}
```

In [32]:
@torch.no_grad()
def extract_logits(model, dataloader):
  logits_buffer = []
  label_buffer = []

  # Follow the above definition of the test_step
  for batch in dataloader:
    audio_signal, audio_signal_len, labels, labels_len = batch
    logits = model(input_signal=audio_signal, input_signal_length=audio_signal_len)

    logits_buffer.append(logits)
    label_buffer.append(labels)
    print(".", end='')
  print()
  
  print("Finished extracting logits !")
  logits = torch.cat(logits_buffer, 0)
  labels = torch.cat(label_buffer, 0)
  return logits, labels


In [33]:
cpu_model = asr_model.cpu()
cpu_model.eval()
logits, labels = extract_logits(cpu_model, test_dl)
print("Logits:", logits.shape, "Labels :", labels.shape)

......................................................
Finished extracting logits !
Logits: torch.Size([6835, 30]) Labels : torch.Size([6835])


In [34]:
# Compute accuracy - `_accuracy` is a PyTorch Lightning Metric !
acc = cpu_model._accuracy(logits=logits, labels=labels)
print("Accuracy : ", float(acc[0]*100))

Accuracy :  94.39649200439453


## Filtering out incorrect samples
Let us now filter out the incorrectly labeled samples from the total set of samples in the test set

In [35]:
import librosa
import json
import IPython.display as ipd

In [36]:
# First let's create a utility class to remap the integer class labels to actual string label
class ReverseMapLabel:
    def __init__(self, data_loader):
        self.label2id = dict(data_loader.dataset.label2id)
        self.id2label = dict(data_loader.dataset.id2label)

    def __call__(self, pred_idx, label_idx):
        return self.id2label[pred_idx], self.id2label[label_idx]

In [37]:
# Next, let's get the indices of all the incorrectly labeled samples
sample_idx = 0
incorrect_preds = []
rev_map = ReverseMapLabel(test_dl)

# Remember, evaluated_tensor = (loss, logits, labels)
probs = torch.softmax(logits, dim=-1)
probas, preds = torch.max(probs, dim=-1)

total_count = cpu_model._accuracy.total_counts_k[0]
incorrect_ids = (preds != labels).nonzero()
for idx in incorrect_ids:
    proba = float(probas[idx][0])
    pred = int(preds[idx][0])
    label = int(labels[idx][0])
    idx = int(idx[0]) + sample_idx

    incorrect_preds.append((idx, *rev_map(pred, label), proba))

print(f"Num test samples : {total_count.item()}")
print(f"Num errors : {len(incorrect_preds)}")

# First lets sort by confidence of prediction
incorrect_preds = sorted(incorrect_preds, key=lambda x: x[-1], reverse=False)

Num test samples : 6835
Num errors : 383


## Examine a subset of incorrect samples
Let's print out the (test id, predicted label, ground truth label, confidence) tuple of first 20 incorrectly labeled samples

In [38]:
for incorrect_sample in incorrect_preds[:20]:
    print(str(incorrect_sample))

(3594, 'two', 'six', 0.053209662437438965)
(3929, 'three', 'marvin', 0.056877534836530685)
(4741, 'up', 'three', 0.05832511931657791)
(1101, 'up', 'five', 0.05911928042769432)
(5668, 'up', 'eight', 0.06363773345947266)
(5644, 'left', 'eight', 0.0697454959154129)
(392, 'on', 'yes', 0.07077573984861374)
(3160, 'on', 'stop', 0.07360755652189255)
(3216, 'yes', 'stop', 0.07615386694669724)
(3993, 'two', 'down', 0.07615914195775986)
(3401, 'up', 'two', 0.07648081332445145)
(17, 'two', 'right', 0.0771590992808342)
(3695, 'up', 'six', 0.07731562852859497)
(2388, 'off', 'up', 0.08139512687921524)
(762, 'off', 'left', 0.08150078356266022)
(6122, 'left', 'zero', 0.08416656404733658)
(5238, 'up', 'four', 0.08515415340662003)
(2920, 'up', 'go', 0.08566978573799133)
(6090, 'four', 'zero', 0.08606620132923126)
(4588, 'three', 'sheila', 0.08793909102678299)


##  Define a threshold below which we designate a model's prediction as "low confidence"

In [39]:
# Filter out how many such samples exist
low_confidence_threshold = 0.25
count_low_confidence = len(list(filter(lambda x: x[-1] <= low_confidence_threshold, incorrect_preds)))
print(f"Number of low confidence predictions : {count_low_confidence}")

Number of low confidence predictions : 82


## Let's hear the samples which the model has least confidence in !

In [40]:
# First let's create a helper function to parse the manifest files
def parse_manifest(manifest):
    data = []
    for line in manifest:
        line = json.loads(line)
        data.append(line)

    return data

In [41]:
# Next, let's create a helper function to actually listen to certain samples
def listen_to_file(sample_id, pred=None, label=None, proba=None):
    # Load the audio waveform using librosa
    filepath = test_samples[sample_id]['audio_filepath']
    audio, sample_rate = librosa.load(filepath)

    if pred is not None and label is not None and proba is not None:
        print(f"Sample : {sample_id} Prediction : {pred} Label : {label} Confidence = {proba: 0.4f}")
    else:
        print(f"Sample : {sample_id}")

    return ipd.Audio(audio, rate=sample_rate)

In [42]:
# Now let's load the test manifest into memory
test_samples = []
with open(test_dataset, 'r') as test_f:
    test_samples = test_f.readlines()

test_samples = parse_manifest(test_samples)

In [43]:
# Finally, let's listen to all the audio samples where the model made a mistake
# Note: This list of incorrect samples may be quite large, so you may choose to subsample `incorrect_preds`
count = min(count_low_confidence, 20)  # replace this line with just `count_low_confidence` to listen to all samples with low confidence

for sample_id, pred, label, proba in incorrect_preds[:count]:
    ipd.display(listen_to_file(sample_id, pred=pred, label=label, proba=proba))

Sample : 3594 Prediction : two Label : six Confidence =  0.0532


Sample : 3929 Prediction : three Label : marvin Confidence =  0.0569


Sample : 4741 Prediction : up Label : three Confidence =  0.0583


Sample : 1101 Prediction : up Label : five Confidence =  0.0591


Sample : 5668 Prediction : up Label : eight Confidence =  0.0636


...

Sample : 6122 Prediction : left Label : zero Confidence =  0.0842


Sample : 5238 Prediction : up Label : four Confidence =  0.0852


Sample : 2920 Prediction : up Label : go Confidence =  0.0857


Sample : 6090 Prediction : four Label : zero Confidence =  0.0861


Sample : 4588 Prediction : three Label : sheila Confidence =  0.0879


# Fine-tuning on a new dataset

We currently trained our dataset on all 30/35 classes of the Google Speech Commands dataset (v1/v2).

We will now show an example of fine-tuning a trained model on a subset of the classes, as a demonstration of fine-tuning.


## Preparing the data-subsets

Let's select 2 of the classes, `yes` and `no` and prepare our manifests with this dataset.

In [44]:
import json

In [45]:
def extract_subset_from_manifest(name: str, manifest_path: str, labels: list):
  manifest_dir = os.path.split(manifest_path)[0]
  labels = set(labels)
  manifest_values = []

  print(f"Parsing manifest: {manifest_path}")
  with open(manifest_path, 'r') as f:
    for line in f:
      val = json.loads(line)

      if val['command'] in labels:
        manifest_values.append(val)

  print(f"Number of files extracted from dataset: {len(manifest_values)}")

  outpath = os.path.join(manifest_dir, name)
  with open(outpath, 'w') as f:
    for val in manifest_values:
      json.dump(val, f)
      f.write("\n")
      f.flush()

  print("Manifest subset written to path :", outpath)
  print()

  return outpath

In [46]:
labels = ["yes", "no"]

train_subdataset = extract_subset_from_manifest("train_subset.json", train_dataset, labels)
val_subdataset = extract_subset_from_manifest("val_subset.json", val_dataset, labels)
test_subdataset = extract_subset_from_manifest("test_subset.json", test_dataset, labels)

Parsing manifest: ./google_dataset_v1/google_speech_recognition_v1/train_manifest.json
Number of files extracted from dataset: 3713
Manifest subset written to path : ./google_dataset_v1/google_speech_recognition_v1/train_subset.json

Parsing manifest: ./google_dataset_v1/google_speech_recognition_v1/validation_manifest.json
Number of files extracted from dataset: 531
Manifest subset written to path : ./google_dataset_v1/google_speech_recognition_v1/val_subset.json

Parsing manifest: ./google_dataset_v1/google_speech_recognition_v1/test_manifest.json
Number of files extracted from dataset: 508
Manifest subset written to path : ./google_dataset_v1/google_speech_recognition_v1/test_subset.json



## Saving/Restoring a checkpoint

There are multiple ways to save and load models in NeMo. Since all NeMo models are inherently Lightning Modules, we can use the standard way that PyTorch Lightning saves and restores models.

NeMo also provides a more advanced model save/restore format, which encapsulates all the parts of the model that are required to restore that model for immediate use.

In this example, we will explore both ways of saving and restoring models, but we will focus on the PyTorch Lightning method.

### Saving and Restoring via PyTorch Lightning Checkpoints

When using NeMo for training, it is advisable to utilize the `exp_manager` framework. It is tasked with handling checkpointing and logging (Tensorboard as well as WandB optionally!), as well as dealing with multi-node and multi-GPU logging.

Since we utilized the `exp_manager` framework above, we have access to the directory where the checkpoints exist. 

`exp_manager` with the default settings will save multiple checkpoints for us - 

1) A few checkpoints from certain steps of training. They will have `--val_loss=` tags

2) A checkpoint at the last epoch of training denotes by `-last`.

3) If the model finishes training, it will also have a `--end` checkpoint.

In [47]:
import glob

In [48]:
print(exp_dir)

/workspace/nemo/tutorials/asr/nemo_experiments/MatchboxNet-3x1x64-v1/2021-11-29_00-55-29


In [49]:
# Let's list all the checkpoints we have
checkpoint_dir = os.path.join(exp_dir, 'checkpoints')
checkpoint_paths = list(glob.glob(os.path.join(checkpoint_dir, "*.ckpt")))
checkpoint_paths

['/workspace/nemo/tutorials/asr/nemo_experiments/MatchboxNet-3x1x64-v1/2021-11-29_00-55-29/checkpoints/MatchboxNet-3x1x64-v1--val_loss=0.2391-epoch=3.ckpt',
 '/workspace/nemo/tutorials/asr/nemo_experiments/MatchboxNet-3x1x64-v1/2021-11-29_00-55-29/checkpoints/MatchboxNet-3x1x64-v1--val_loss=0.3785-epoch=2.ckpt',
 '/workspace/nemo/tutorials/asr/nemo_experiments/MatchboxNet-3x1x64-v1/2021-11-29_00-55-29/checkpoints/MatchboxNet-3x1x64-v1--val_loss=0.2031-epoch=4.ckpt',
 '/workspace/nemo/tutorials/asr/nemo_experiments/MatchboxNet-3x1x64-v1/2021-11-29_00-55-29/checkpoints/MatchboxNet-3x1x64-v1--val_loss=0.2031-epoch=4-last.ckpt']

In [50]:
# We want the checkpoint saved after the final step of training
final_checkpoint = list(filter(lambda x: "-last.ckpt" in x, checkpoint_paths))[0]
print(final_checkpoint)

/workspace/nemo/tutorials/asr/nemo_experiments/MatchboxNet-3x1x64-v1/2021-11-29_00-55-29/checkpoints/MatchboxNet-3x1x64-v1--val_loss=0.2031-epoch=4-last.ckpt


### Restoring from a PyTorch Lightning checkpoint

To restore a model using the `LightningModule.load_from_checkpoint()` class method.

In [51]:
restored_model = nemo_asr.models.EncDecClassificationModel.load_from_checkpoint(final_checkpoint)

[NeMo W 2021-11-29 01:01:53 modelPT:130] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: ./google_dataset_v1/google_speech_recognition_v1/train_manifest.json
    sample_rate: 16000
    labels:
    - bed
    - bird
    - cat
    - dog
    - down
    - eight
    - five
    - four
    - go
    - happy
    - house
    - left
    - marvin
    - nine
    - 'no'
    - 'off'
    - 'on'
    - one
    - right
    - seven
    - sheila
    - six
    - stop
    - three
    - tree
    - two
    - up
    - wow
    - 'yes'
    - zero
    batch_size: 128
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: null
    tarred_shard_strategy: scatter
    augmentor:
      shift:
        prob: 1.0
        min_shift_ms: -5.0
        max_shift_ms: 5.0
      white_noise:
        prob: 1.0
        min_level: -90
        max_level: -46
   

## Prepare the model for fine-tuning

Remember, the original model was trained for a 30/35 way classification task. Now we require only a subset of these models, so we need to modify the decoder head to support fewer classes.

We can do this easily with the convenient function `EncDecClassificationModel.change_labels(new_label_list)`.

By performing this step, we discard the old decoder head, but still, preserve the encoder!

In [52]:
restored_model.change_labels(labels)

[NeMo I 2021-11-29 01:01:53 classification_models:609] Changed decoder output to 2 labels.


### Prepare the data loaders

The restored model, upon restoration, will not attempt to set up any data loaders. 

This is so that we can manually set up any datasets we want - train and val to finetune the model, test in order to just evaluate, or all three to do both!

The entire config that we used before can still be accessed via `ModelPT.cfg`, so we will use it in order to set up our data loaders. This also gives us the opportunity to set any additional parameters we wish to setup!

In [53]:
import copy

In [54]:
train_subdataset_cfg = copy.deepcopy(restored_model.cfg.train_ds)
val_subdataset_cfg = copy.deepcopy(restored_model.cfg.validation_ds)
test_subdataset_cfg = copy.deepcopy(restored_model.cfg.test_ds)

In [55]:
# Set the paths to the subset of the dataset
train_subdataset_cfg.manifest_filepath = train_subdataset
val_subdataset_cfg.manifest_filepath = val_subdataset
test_subdataset_cfg.manifest_filepath = test_subdataset

In [56]:
# Setup the data loader for the restored model
restored_model.setup_training_data(train_subdataset_cfg)
restored_model.setup_multiple_validation_data(val_subdataset_cfg)
restored_model.setup_multiple_test_data(test_subdataset_cfg)

[NeMo I 2021-11-29 01:01:53 collections:266] Filtered duration for loading collection is 0.000000.
[NeMo I 2021-11-29 01:01:53 collections:270] # 3713 files loaded accounting to # 2 labels
[NeMo I 2021-11-29 01:01:53 collections:266] Filtered duration for loading collection is 0.000000.
[NeMo I 2021-11-29 01:01:53 collections:270] # 531 files loaded accounting to # 2 labels
[NeMo I 2021-11-29 01:01:53 collections:266] Filtered duration for loading collection is 0.000000.
[NeMo I 2021-11-29 01:01:53 collections:270] # 508 files loaded accounting to # 2 labels


In [57]:
# Check data loaders are correct
print("Train dataset labels :", restored_model._train_dl.dataset.labels)
print("Val dataset labels :", restored_model._validation_dl.dataset.labels)
print("Test dataset labels :", restored_model._test_dl.dataset.labels)

Train dataset labels : ['yes', 'no']
Val dataset labels : ['yes', 'no']
Test dataset labels : ['yes', 'no']


## Setting up a new Trainer and Experiment Manager

A restored model has a utility method to attach the Trainer object to it, which is necessary in order to correctly set up the optimizer and scheduler!

**Note**: The restored model does not contain the trainer config with it. It is necessary to create a new Trainer object suitable for the environment where the model is being trained. The template can be replicated from any of the training scripts.

Here, since we already had the previous config object that prepared the trainer, we could have used it, but for demonstration, we will set up the trainer config manually.

In [58]:
# Setup the new trainer object
# Let's modify some trainer configs for this demo
# Checks if we have GPU available and uses it
cuda = 1 if torch.cuda.is_available() else 0

trainer_config = OmegaConf.create(dict(
    gpus=cuda,
    max_epochs=5,
    max_steps=None,  # computed at runtime if not set
    num_nodes=1,
    accumulate_grad_batches=1,
    checkpoint_callback=False,  # Provided by exp_manager
    logger=False,  # Provided by exp_manager
    log_every_n_steps=1,  # Interval of logging.
    val_check_interval=1.0,  # Set to 0.25 to check 4 times per epoch, or an int for number of iterations
))
print(OmegaConf.to_yaml(trainer_config))

gpus: 1
max_epochs: 5
max_steps: null
num_nodes: 1
accumulate_grad_batches: 1
checkpoint_callback: false
logger: false
log_every_n_steps: 1
val_check_interval: 1.0



In [59]:
trainer_finetune = pl.Trainer(**trainer_config)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


### Setting the trainer to the restored model

All NeMo models provide a convenience method `set_trainer()` in order to setup the trainer after restoration

In [60]:
restored_model.set_trainer(trainer_finetune)

In [61]:
exp_dir_finetune = exp_manager(trainer_finetune, config.get("exp_manager", None))

[NeMo I 2021-11-29 01:01:53 exp_manager:220] Experiments will be logged at /workspace/nemo/tutorials/asr/nemo_experiments/MatchboxNet-3x1x64-v1/2021-11-29_00-55-29
[NeMo I 2021-11-29 01:01:53 exp_manager:569] TensorboardLogger has been set up


[NeMo W 2021-11-29 01:01:53 nemo_logging:349] /opt/conda/lib/python3.8/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:240: LightningDeprecationWarning: `ModelCheckpoint(every_n_val_epochs)` is deprecated in v1.4 and will be removed in v1.6. Please use `every_n_epochs` instead.
      rank_zero_deprecation(
    
[NeMo W 2021-11-29 01:01:53 nemo_logging:349] /opt/conda/lib/python3.8/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:446: UserWarning: Checkpoint directory /workspace/nemo/tutorials/asr/nemo_experiments/MatchboxNet-3x1x64-v1/2021-11-29_00-55-29/checkpoints exists and is not empty.
      rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
    


In [62]:
exp_dir_finetune = str(exp_dir_finetune)
exp_dir_finetune

'/workspace/nemo/tutorials/asr/nemo_experiments/MatchboxNet-3x1x64-v1/2021-11-29_00-55-29'

## Setup optimizer + scheduler

For a fine-tuning experiment, let's set up the optimizer and scheduler!

We will use a much lower learning rate than before, and also swap out the scheduler from PolyHoldDecay to CosineDecay.

In [63]:
optim_sched_cfg = copy.deepcopy(restored_model.cfg.optim)
# Struct mode prevents us from popping off elements from the config, so let's disable it
OmegaConf.set_struct(optim_sched_cfg, False)

In [64]:
# Lets change the maximum learning rate to previous minimum learning rate
optim_sched_cfg.lr = 0.001

# Lets change the scheduler
optim_sched_cfg.sched.name = "CosineAnnealing"

# "power" isnt applicable to CosineAnnealing so let's remove it
optim_sched_cfg.sched.pop('power')

# "hold_ratio" isnt applicable to CosineAnnealing, so let's remove it
optim_sched_cfg.sched.pop('hold_ratio')

# Set "min_lr" to lower value
optim_sched_cfg.sched.min_lr = 1e-4

print(OmegaConf.to_yaml(optim_sched_cfg))

name: novograd
lr: 0.001
betas:
- 0.95
- 0.5
weight_decay: 0.001
sched:
  name: CosineAnnealing
  warmup_ratio: 0.05
  min_lr: 0.0001
  last_epoch: -1



In [65]:
# Now lets update the optimizer settings
restored_model.setup_optimization(optim_sched_cfg)

[NeMo I 2021-11-29 01:01:53 modelPT:544] Optimizer config = Novograd (
    Parameter Group 0
        amsgrad: False
        betas: [0.95, 0.5]
        eps: 1e-08
        grad_averaging: False
        lr: 0.001
        weight_decay: 0.001
    )
[NeMo I 2021-11-29 01:01:53 lr_scheduler:621] Scheduler "<nemo.core.optim.lr_scheduler.CosineAnnealing object at 0x7f732c54b640>" 
    will be used during training (effective maximum steps = 150) - 
    Parameters : 
    (warmup_ratio: 0.05
    min_lr: 0.0001
    last_epoch: -1
    max_steps: 150
    )


(Novograd (
 Parameter Group 0
     amsgrad: False
     betas: [0.95, 0.5]
     eps: 1e-08
     grad_averaging: False
     initial_lr: 0.001
     lr: 0.000125
     weight_decay: 0.001
 ),
 {'scheduler': <nemo.core.optim.lr_scheduler.CosineAnnealing at 0x7f732c54b640>,
  'interval': 'step',
  'frequency': 1,
  'monitor': 'loss',
  'reduce_on_plateau': False})

In [66]:
# We can also just directly replace the config inplace if we choose to
restored_model.cfg.optim = optim_sched_cfg

## Fine-tune training step

We fine-tune on the subset classification problem. Note, the model was originally trained on these classes (the subset defined here has already been trained on above).

When fine-tuning on a truly new dataset, we will not see such a dramatic improvement in performance. However, it should still converge a little faster than if it was trained from scratch.

### Monitor training progress via Tensorboard


In [67]:
if COLAB_ENV:
  %tensorboard --logdir {exp_dir_finetune}
else:
  print("To use tensorboard, please use this notebook in a Google Colab environment.")

To use tensorboard, please use this notebook in a Google Colab environment.


In [68]:
%tensorboard --logdir {exp_dir_finetune}

Reusing TensorBoard on port 6006 (pid 768), started 0:06:22 ago. (Use '!kill 768' to kill it.)

### Fine-tuning for 5 epochs

In [69]:
%%time
trainer_finetune.fit(restored_model)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


[NeMo I 2021-11-29 01:01:53 modelPT:544] Optimizer config = Novograd (
    Parameter Group 0
        amsgrad: False
        betas: [0.95, 0.5]
        eps: 1e-08
        grad_averaging: False
        lr: 0.001
        weight_decay: 0.001
    )
[NeMo I 2021-11-29 01:01:53 lr_scheduler:621] Scheduler "<nemo.core.optim.lr_scheduler.CosineAnnealing object at 0x7f732c5e2a60>" 
    will be used during training (effective maximum steps = 150) - 
    Parameters : 
    (warmup_ratio: 0.05
    min_lr: 0.0001
    last_epoch: -1
    max_steps: 150
    )



  | Name              | Type                             | Params
-----------------------------------------------------------------------
0 | spec_augmentation | SpectrogramAugmentation          | 0     
1 | crop_or_pad       | CropOrPadSpectrogramAugmentation | 0     
2 | preprocessor      | AudioToMFCCPreprocessor          | 0     
3 | encoder           | ConvASREncoder                   | 73.3 K
4 | loss              | CrossEntropyLoss                 | 0     
5 | _accuracy         | TopKClassificationAccuracy       | 0     
6 | decoder           | ConvASRDecoderClassification     | 258   
-----------------------------------------------------------------------
73.6 K    Trainable params
0         Non-trainable params
73.6 K    Total params
0.294     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

[NeMo W 2021-11-29 01:01:53 nemo_logging:349] /opt/conda/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:105: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
      rank_zero_warn(
    
[NeMo W 2021-11-29 01:01:53 nemo_logging:349] /opt/conda/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:105: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
      rank_zero_warn(
    


...

Validating: 0it [00:00, ?it/s]

Epoch 3, global step 119: val_loss reached 0.03116 (best 0.03116), saving model to "/workspace/nemo/tutorials/asr/nemo_experiments/MatchboxNet-3x1x64-v1/2021-11-29_00-55-29/checkpoints/MatchboxNet-3x1x64-v1--val_loss=0.0312-epoch=3.ckpt" as top 3


Validating: 0it [00:00, ?it/s]

Epoch 4, global step 149: val_loss reached 0.03267 (best 0.03116), saving model to "/workspace/nemo/tutorials/asr/nemo_experiments/MatchboxNet-3x1x64-v1/2021-11-29_00-55-29/checkpoints/MatchboxNet-3x1x64-v1--val_loss=0.0327-epoch=4.ckpt" as top 3


CPU times: user 26.6 s, sys: 1.03 s, total: 27.7 s
Wall time: 27.5 s


### Evaluation on the Test set

Let's compute the final score on the test set via `trainer.test(model)`

In [70]:
trainer_finetune.test(restored_model, ckpt_path=None)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
[NeMo W 2021-11-29 01:02:21 nemo_logging:349] /opt/conda/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:105: UserWarning: The dataloader, test dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
      rank_zero_warn(
    


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_epoch_top@1': 0.9842519164085388, 'test_loss': 0.03662347048521042}
--------------------------------------------------------------------------------


[{'test_loss': 0.03662347048521042, 'test_epoch_top@1': 0.9842519164085388}]

## Advanced Usage: Exporting a model in its entirety

While most models can be easily serialized via the Experiment Manager as a PyTorch Lightning checkpoint, there are certain models where this is insufficient. 

Consider the case where a Model contains artifacts such as tokenizers or other intermediate file objects that cannot be so easily serialized into a checkpoint.

For such cases, NeMo offers two utility functions that enable serialization of a Model + artifacts - `save_to` and `restore_from`.

Further documentation regarding these methods can be obtained from the documentation pages on NeMo.

In [71]:
import tarfile

In [72]:
# Save a model as a tarfile
restored_model.save_to(os.path.join(exp_dir_finetune, "model.nemo"))

In [73]:
# The above object is just a tarfile which can store additional artifacts.
with tarfile.open(os.path.join(exp_dir_finetune, 'model.nemo')) as blob:
  for item in blob:
    print(item)

<TarInfo '.' at 0x7f732c783c40>
<TarInfo './model_config.yaml' at 0x7f732c783700>
<TarInfo './model_weights.ckpt' at 0x7f732c783e80>


In [74]:
# Restore a model from a tarfile
restored_model_2 = nemo_asr.models.EncDecClassificationModel.restore_from(os.path.join(exp_dir_finetune, "model.nemo"))

[NeMo W 2021-11-29 01:02:21 modelPT:130] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: ./google_dataset_v1/google_speech_recognition_v1/train_subset.json
    sample_rate: 16000
    labels:
    - 'yes'
    - 'no'
    batch_size: 128
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: null
    tarred_shard_strategy: scatter
    augmentor:
      shift:
        prob: 1.0
        min_shift_ms: -5.0
        max_shift_ms: 5.0
      white_noise:
        prob: 1.0
        min_level: -90
        max_level: -46
    
[NeMo W 2021-11-29 01:02:21 modelPT:137] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath: ./google

[NeMo I 2021-11-29 01:02:21 save_restore_connector:143] Model EncDecClassificationModel was successfully restored from /workspace/nemo/tutorials/asr/nemo_experiments/MatchboxNet-3x1x64-v1/2021-11-29_00-55-29/model.nemo.


## Conclusion
Once the model has been restored, either via a PyTorch Lightning checkpoint or via the `restore_from` methods, one can finetune by following the above general steps.